In [8]:
import requests
import pandas as pd
import streamlit as st
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from textblob import TextBlob

In [9]:
# 1. Setup Spark Session
# --------------------------
spark = SparkSession.builder \
    .appName("RealTimeNewsSentiment") \
    .getOrCreate()

In [19]:
# 2. Fetch Real-Time News
# --------------------------
def fetch_news(api_key, query="technology", language="en", page_size=20):
    url = f"https://newsapi.org/v2/top-headlines?q={query}&language={language}&pageSize={page_size}&apiKey={api_key}"
    response = requests.get(url)
    data = response.json()
    articles = data.get("articles", [])
    headlines = [a["title"] for a in articles if a["title"]]
    return headlines


In [20]:
# 3. Prepare Training Data
# --------------------------
# Small sample dataset for demonstration
train_data = [
    ("Stocks rally as tech shares soar", "Positive"),
    ("Company reports record profits", "Positive"),
    ("Market crashes amid economic fears", "Negative"),
    ("Political tensions drive uncertainty", "Negative"),
    ("New breakthrough in cancer research", "Positive"),
    ("Floods devastate several regions", "Negative"),
]

train_df = spark.createDataFrame(train_data, ["text", "label"])


In [21]:
# 4. Build Spark ML Pipeline
# --------------------------
tokenizer = Tokenizer(inputCol="text", outputCol="words")
remover = StopWordsRemover(inputCol="words", outputCol="filtered")
hashingTF = HashingTF(inputCol="filtered", outputCol="rawFeatures", numFeatures=1000)
idf = IDF(inputCol="rawFeatures", outputCol="features")
lr = LogisticRegression(labelCol="labelIndex", featuresCol="features")

# StringIndexer to convert labels
from pyspark.ml.feature import StringIndexer
indexer = StringIndexer(inputCol="label", outputCol="labelIndex")

pipeline = Pipeline(stages=[tokenizer, remover, hashingTF, idf, indexer, lr])

# Fit the model
model = pipeline.fit(train_df)

In [22]:
# 5. Sentiment Prediction UDF (Fallback with TextBlob if needed)
# --------------------------
def get_sentiment(text):
    analysis = TextBlob(text).sentiment.polarity
    return "Positive" if analysis >= 0 else "Negative"

sentiment_udf = udf(get_sentiment, StringType())


In [23]:
# 6. Streamlit Dashboard
# --------------------------
def run_dashboard(api_key):
    st.title("📰 Real-Time News Sentiment Dashboard (PySpark + Streamlit)")

    query = st.text_input("Enter topic:", "technology")
    if st.button("Fetch News"):
        headlines = fetch_news(api_key, query=query)

        if not headlines:
            st.warning("No news found.")
            return

        # Convert to Spark DataFrame
        df = spark.createDataFrame([(h,) for h in headlines], ["text"])

        # Predict with PySpark model
        preds = model.transform(df)

        # Map predictions back to Positive/Negative
        label_indexer = dict(enumerate(model.stages[-2].labels))  # from StringIndexer
        preds = preds.withColumn("sentiment", col("prediction").cast("int"))
        preds = preds.withColumn("sentiment", udf(lambda x: label_indexer[x], StringType())(col("sentiment")))

        # Convert to Pandas for dashboard
        pdf = preds.select("text", "sentiment").toPandas()

        st.dataframe(pdf)

        # Sentiment counts
        counts = pdf["sentiment"].value_counts()
        st.bar_chart(counts)

In [24]:
# 7. Run App
# --------------------------
if __name__ == "__main__":
    # 🔑 Replace with your NewsAPI key
    API_KEY = "eefb8c41863048df9e948694f8b52e6a"

    run_dashboard(API_KEY)

2025-09-26 10:35:05.459 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 10:35:05.678 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-09-26 10:35:05.679 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 10:35:05.681 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 10:35:05.683 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 10:35:05.686 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 10:35:05.687 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-26 10:35:05.690 Thread 'MainThread': mi